In [1]:
import os
import logging
import pandas as pd
import numpy as np
from collections import defaultdict

In [18]:
def get_sample_files(path,outfile):
    samples = defaultdict(dict)
    seen = set()
    for dir_name, sub_dirs, files in os.walk(os.path.abspath(path)):
        for fname in files:

            if ".fastq" in fname or ".fq" in fname:

                sample_id = fname.split(".fastq")[0].split(".fq")[0]

                sample_id = sample_id.replace("_R1", "").replace("_r1", "").replace("_R2", "").replace("_r2", "")
                #sample_id = sample_id.replace("_", "-").replace(" ", "-")

                fq_path = os.path.join(dir_name, fname)

                if fq_path in seen: continue

                if "_R2" in fname or "_r2" in fname:

                    if 'R2' in samples[sample_id]:
                        logging.error(f"Duplicate sample {sample_id} was found after renaming; skipping... \n Samples: \n{samples}")

                    samples[sample_id]['R2'] = fq_path
                else:
                    if 'R1' in samples[sample_id]:
                        logging.error(f"Duplicate sample {sample_id} was found after renaming; skipping... \n Samples: \n{samples}")

                    samples[sample_id]['R1'] = fq_path


    samples= pd.DataFrame(samples).T
    samples['GROUP']=samples.index.to_series().apply(lambda x: int(x.split("-")[1]) < 101).map({True: 'lib1', False:'lib2'})

    if samples.isna().any().any():
        logging.error(f"Missing files:\n {samples}")

    #if os.path.exists(outfile):
        #logging.error(f"Output file {outfile} already exists I don't dare to overwrite it.")
        #exit(1)
    else:
        samples.to_csv(outfile,sep='\t')


    return samples

In [19]:
D = get_sample_files("/home/lvelo/working_directory/microbiology/METEL/00_raw/","METEL.tsv")
D

R1  \
AUPO-177  /home/lvelo/working_directory/microbiology/MET...   
AUPO-176  /home/lvelo/working_directory/microbiology/MET...   
AUPO-100  /home/lvelo/working_directory/microbiology/MET...   
AUPO-99   /home/lvelo/working_directory/microbiology/MET...   

                                                         R2 GROUP  
AUPO-177  /home/lvelo/working_directory/microbiology/MET...  lib2  
AUPO-176  /home/lvelo/working_directory/microbiology/MET...  lib2  
AUPO-100  /home/lvelo/working_directory/microbiology/MET...  lib1  
AUPO-99   /home/lvelo/working_directory/microbiology/MET...  lib1